This notebook contains a few examples for bcc Fe to illustrate how pyVASP works

- Note: They only show how to create the inputs (set_calculation()), to submit the job, use:
   
   self.run_vasp() or self.submit_job()

In [1]:
import os
from subprocess import run
import numpy as np
from pyVASP.code.main import pyVASP

import pandas as pd
import ase
from ase.spacegroup import crystal

# Define a custom formatting function for vectors in pandas
format_vector = lambda vector: '[' + ', '.join(['{:.10f}'.format(x) for x in vector]) + ']'

import nglview

In [2]:
dir_0 = os.getcwd()

In [3]:
# set executable
executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"

# simple FM bcc Fe

set working directory

In [4]:
cwd = dir_0 + "/FM"
if not os.path.exists(cwd):
   run("mkdir " + cwd, shell=True)

set crystal structure

In [5]:
a = 2.79 # A
alpha = 90
structure = crystal(('Fe'),
                       basis=[(0.00, 0.00, 0.00)],
                       spacegroup=229,
                       cellpar=[a, a, a, alpha, alpha, alpha])

set calculation

In [6]:
# create vasp class
vasp = pyVASP(verbose="low", executable_path=executable_path, potential_path=potential_path)

# update current working directory
vasp.io.cwd = cwd

In [7]:
# set magdirs for a FM state
number_of_atoms = len(structure)
magdirs   = []
for i in range(number_of_atoms):
   if 'Fe' == structure.get_chemical_symbols()[i]:
      magdirs.append([0, 0, 1])

structure.new_array("magdirs", magdirs, dtype=float)

In [8]:
# set calculation, which creates all input files
vasp.set_calculation(structure)

visualize structure

In [9]:
view = nglview.show_ase(structure)
view.add_unitcell()
view

NGLWidget()

visualize some inputs

In [10]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
0,Fe,"(0.0, 0.0, 0.0)","(0.0, 0.0, 1.0)",1.0,inf,"(0.0, 0.0, 1.0)","(0.0, 0.0, 0.0)"
1,Fe,"(1.395, 1.395, 1.395)","(0.0, 0.0, 1.0)",1.0,inf,"(0.0, 0.0, 1.0)","(0.0, 0.0, 0.0)"


In [11]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

0    [0.0000000000, 0.0000000000, 1.0000000000]
1    [0.0000000000, 0.0000000000, 1.0000000000]
Name: magmoms, dtype: object

In [12]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 12 12 12

lattice vectors =
 [[2.79 0.   0.  ]
 [0.   2.79 0.  ]
 [0.   0.   2.79]]

species = ['Fe']

elements = 0    Fe
1    Fe
Name: elements, dtype: object


# FM bcc Fe, using a primitive cell, and relax structure

set working directory

In [13]:
cwd = dir_0 + "/FM_primitive_cell_relax"
if not os.path.exists(cwd):
   run("mkdir " + cwd, shell=True)

set crystal structure

In [14]:
a = 2.79 # A
alpha = 90
structure = crystal(('Fe'),
                       basis=[(0.00, 0.00, 0.00)],
                       spacegroup=229,
                       primitive_cell=True,
                       cellpar=[a, a, a, alpha, alpha, alpha])

set calculation

In [15]:
# create vasp class
vasp = pyVASP(verbose="low", executable_path=executable_path, potential_path=potential_path)

# update current working directory
vasp.io.cwd = cwd

In [16]:
# Set magdirs for a FM state
number_of_atoms = len(structure)
magdirs   = []
for i in range(number_of_atoms):
   if 'Fe' == structure.get_chemical_symbols()[i]:
      magdirs.append([0, 0, 1])

structure.new_array("magdirs", magdirs, dtype=float)

In [17]:
# first prepare relaxation
vasp.prepare_relaxation(ISIF="3")
# set calculation, which creates all input files
vasp.set_calculation(structure)

visualize structure

In [18]:
view = nglview.show_ase(structure)
view.add_unitcell()
view

NGLWidget()

visualize some inputs

In [19]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
0,Fe,"(0.0, 0.0, 0.0)","(0.0, 0.0, 1.0)",1.0,inf,"(0.0, 0.0, 1.0)","(0.0, 0.0, 0.0)"


In [20]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

0    [0.0000000000, 0.0000000000, 1.0000000000]
Name: magmoms, dtype: object

In [21]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 12 12 12

lattice vectors =
 [[-1.395  1.395  1.395]
 [ 1.395 -1.395  1.395]
 [ 1.395  1.395 -1.395]]

species = ['Fe']

elements = 0    Fe
Name: elements, dtype: object


# AFM bcc Fe and change kpoints and ntasks

set working directory

In [22]:
cwd = dir_0 + "/AFM_kpoints10x10x10_ntasks80"
if not os.path.exists(cwd):
   run("mkdir " + cwd, shell=True)

set crystal structure

In [23]:
a = 2.79 # A
alpha = 90
structure = crystal(('Fe'),
                       basis=[(0.00, 0.00, 0.00)],
                       spacegroup=229,
                       cellpar=[a, a, a, alpha, alpha, alpha])

set calculation

In [24]:
# create vasp class
vasp = pyVASP(verbose="low", executable_path=executable_path, potential_path=potential_path)

# update current working directory
vasp.io.cwd = cwd

# update kpoints
vasp.structure.kpoints = "10 10 10"

In [25]:
# Set magdirs for a AFM state
magdirs   = []
magdirs.append([0, 0, 1])
magdirs.append([0, 0, -1])

structure.new_array("magdirs", magdirs, dtype=float)

In [26]:
# first prepare relaxation
vasp.prepare_relaxation(ISIF="3")
# set calculation, which creates all input files
vasp.set_calculation(structure, ntasks=80)

visualize some inputs

In [27]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
1,Fe,"(1.395, 1.395, 1.395)","(0.0, 0.0, -1.0)",1.0,inf,"(0.0, 0.0, -1.0)","(0.0, 0.0, 0.0)"
0,Fe,"(0.0, 0.0, 0.0)","(0.0, 0.0, 1.0)",1.0,inf,"(0.0, 0.0, 1.0)","(0.0, 0.0, 0.0)"


In [28]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

1    [0.0000000000, 0.0000000000, -1.0000000000]
0     [0.0000000000, 0.0000000000, 1.0000000000]
Name: magmoms, dtype: object

In [29]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 10 10 10

lattice vectors =
 [[2.79 0.   0.  ]
 [0.   2.79 0.  ]
 [0.   0.   2.79]]

species = ['Fe']

elements = 1    Fe
0    Fe
Name: elements, dtype: object


In [30]:
vasp.io.write_fields(vasp.io.INCAR)

   SYSTEM = job
   PREC = Accurate
   ALGO = Fast
   ISTART = 0
   ICHARG = 2
   LREAL = .FALSE.
   LWAVE = .FALSE.
   LCHARG = .TRUE.
   LORBIT = 11
   ISMEAR = 1
   SIGMA = 0.005
   ISPIN = 2
   ENCUT = 500
   EDIFF = 1e-6
   NELM = 200
   LMAXMIX = 6
   LNONCOLLINEAR = .TRUE.
   NPAR = 8
   ISYM = 2


In [31]:
vasp.io.write_fields(vasp.io.job_parameters)

   job_name = job
   partition = p.cmfe
   nodes = 2
   ntasks = 40
   constraint = '[swi1|swi2|swi3|swi4|swi5|swi6|swi7|swe1|swe2|swe3|swe4|swe5|swe6|swe7]'
   time = 180
   mem_per_cpu = 3GB
   output = mpi-out.%j
   error = mpi-err.%j
   get_user_env = L


# DLM bcc Fe

set working directory

In [32]:
cwd = dir_0 + "/DLM"
if not os.path.exists(cwd):
   run("mkdir " + cwd, shell=True)

set crystal structure

In [33]:
a = 2.79 # A
alpha = 90
structure = crystal(('Fe'),
                       basis=[(0.00, 0.00, 0.00)],
                       spacegroup=229,
                       cellpar=[a, a, a, alpha, alpha, alpha])

set calculation

In [34]:
# create vasp class
vasp = pyVASP(verbose="low", executable_path=executable_path, potential_path=potential_path, seed_mag=23)

# update current working directory
vasp.io.cwd = cwd

In [35]:
value_of_m = 0.0

number_of_atoms = len(structure)
magdirs   = []
ms    = []
for i in range(number_of_atoms):
   if 'Fe' == structure.get_chemical_symbols()[i]:
      magdirs.append([0, 0, 1])
      ms.append(value_of_m)

structure.new_array("magdirs", magdirs, dtype=float)
structure.new_array("ms", ms, dtype=float)

In [36]:
# first prepare bfields (constraints)
vasp.prepare_bfields(I_CONSTRAINED_M="4", LAMBDA="1")
# set calculation, which creates all input files
vasp.set_calculation(structure)

visualize some inputs

In [37]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
0,Fe,"(0.0, 0.0, 0.0)","(0.0, 0.0, 1.0)",0.0,0.0,"(0.9444205603370265, -0.3269142671564162, -0.0...","(0.0, 0.0, 0.0)"
1,Fe,"(1.395, 1.395, 1.395)","(0.0, 0.0, 1.0)",0.0,0.0,"(-0.17130337138457336, 0.8299274784678624, -0....","(0.0, 0.0, 0.0)"


In [38]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

0    [0.9444205603, -0.3269142672, -0.0345957677]
1    [-0.1713033714, 0.8299274785, -0.5309195188]
Name: magmoms, dtype: object

In [39]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 12 12 12

lattice vectors =
 [[2.79 0.   0.  ]
 [0.   2.79 0.  ]
 [0.   0.   2.79]]

species = ['Fe']

elements = 0    Fe
1    Fe
Name: elements, dtype: object


# Show default values

In [40]:
# create vasp class
vasp = pyVASP(verbose="high", executable_path=executable_path, potential_path=potential_path)


Your executable is:
   /home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin/vasp_ncl

Your potential path is:
   /home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE/

Your current working directory (cwd) is:
   /home/emendive/workbench/devel/gitclones/pyVASP/notebooks/examples/bccFe_FM/

Your bfields and relaxation flags are:
   bfields = False
   relaxation = False

Your seed_mag for DLM approach is:
   seed_mag = random

Your default INCAR parameters are:
   SYSTEM = job
   PREC = Accurate
   ALGO = Fast
   ISTART = 0
   ICHARG = 2
   LREAL = .FALSE.
   LWAVE = .FALSE.
   LCHARG = .TRUE.
   LORBIT = 11
   ISMEAR = 1
   SIGMA = 0.005
   ISPIN = 2
   ENCUT = 500
   EDIFF = 1e-6
   NELM = 200
   LMAXMIX = 6
   LNONCOLLINEAR = .TRUE.
   NPAR = 5
   ISYM = 2

Your default INCAR parameters for constraining fields are:
   I_CONSTRAINED_M = 4
   LAMBDA = 1
   B_MIX = 1.0
   B_ref = 0.02
   N_MIX = 4
   E_PENALTY_MAX = 3.8
   LAMBDA_FIELD_MAX = 1e-3
